In [5]:
!gdown 1XGLAnvZt9oEYCscQcqHjbva0k5S605BZ
!gdown 1EEBfIUZD0q0wRXo8pPFnIGzlzaexI6si

Downloading...
From: https://drive.google.com/uc?id=1XGLAnvZt9oEYCscQcqHjbva0k5S605BZ
To: /content/test_data.csv
100% 176k/176k [00:00<00:00, 94.7MB/s]
Downloading...
From: https://drive.google.com/uc?id=1EEBfIUZD0q0wRXo8pPFnIGzlzaexI6si
To: /content/train_data.csv
100% 692k/692k [00:00<00:00, 115MB/s]


In [6]:
import numpy as np
import pandas as pd
import gensim
import gensim.downloader as api
from sklearn.naive_bayes import MultinomialNB, BernoulliNB, GaussianNB
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_score, recall_score, f1_score
from xgboost import XGBClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV

In [7]:
train_df = pd.read_csv("train_data.csv")
X_train = train_df['clean_text'].to_numpy()
y_train = train_df['target'].to_numpy()

In [8]:
test_df = pd.read_csv("test_data.csv")
X_test = test_df['clean_text'].to_numpy()
y_test = test_df['target'].to_numpy()

# Pretrained Word2Vec

In [9]:
path = api.load("word2vec-google-news-300", return_path=True)
print(path)

[==================================================] 100.0% 1662.8/1662.8MB downloaded
/root/gensim-data/word2vec-google-news-300/word2vec-google-news-300.gz


In [18]:
model_w2v = gensim.models.KeyedVectors.load_word2vec_format(path, binary=True)

In [19]:
voc = set(model_w2v.index_to_key)
def document_vector(doc):
    doc = doc.split()
    doc = [word for word in doc if word in voc]
    return np.mean(model_w2v[doc], axis=0)

In [20]:
X_train_w2v = []
for review in X_train:
    X_train_w2v.append(document_vector(review))

In [21]:
X_test_w2v = []
for review in X_test:
    X_test_w2v.append(document_vector(review))

In [22]:
np.array(X_train_w2v).shape

(2842, 300)

# Training and Testing

In [23]:
def training_and_prediction(training_model, X_train, X_test, y_train, y_test):
  model = training_model
  model.fit(X_train, y_train)
  y_pred = model.predict(X_test)
  accuracy = accuracy_score(y_test, y_pred)
  precision = precision_score(y_test, y_pred)
  recall = recall_score(y_test, y_pred)
  f1score = f1_score(y_test, y_pred)
  # print("Accuracy =", accuracy)
  # print("Confusion Matrix =\n", confusion_matrix(y_test, y_pred))
  # print(classification_report(y_test, y_pred))
  return {'Accuracy' : accuracy, 'Precision': precision, 'Recall' : recall, 'F1-Score' : f1score}

In [24]:
def get_report(y_test, y_pred):
  accuracy = accuracy_score(y_test, y_pred)
  precision = precision_score(y_test, y_pred)
  recall = recall_score(y_test, y_pred)
  f1score = f1_score(y_test, y_pred)
  return {'Accuracy' : accuracy, 'Precision': precision, 'Recall' : recall, 'F1-Score' : f1score}

## Naive Bayes

In [25]:
scaler = MinMaxScaler()
training_and_prediction(MultinomialNB(), scaler.fit_transform(X_train_w2v), scaler.fit_transform(X_test_w2v), y_train, y_test)

{'Accuracy': 0.6849507735583685,
 'Precision': 0.7489711934156379,
 'Recall': 0.527536231884058,
 'F1-Score': 0.619047619047619}

In [26]:
training_and_prediction(GaussianNB(), X_train_w2v, X_test_w2v, y_train, y_test)

{'Accuracy': 0.6624472573839663,
 'Precision': 0.6277372262773723,
 'Recall': 0.7478260869565218,
 'F1-Score': 0.6825396825396826}

In [27]:
training_and_prediction(BernoulliNB(), X_train_w2v, X_test_w2v, y_train, y_test)

{'Accuracy': 0.6343178621659634,
 'Precision': 0.6009501187648456,
 'Recall': 0.7333333333333333,
 'F1-Score': 0.6605744125326369}

## SVM

In [41]:
training_and_prediction(SVC(C=0.8), X_train_w2v, X_test_w2v, y_train, y_test)

{'Accuracy': 0.7426160337552743,
 'Precision': 0.7314285714285714,
 'Recall': 0.7420289855072464,
 'F1-Score': 0.7366906474820144}

## Random Forest

In [42]:
training_and_prediction(RandomForestClassifier(), X_train_w2v, X_test_w2v, y_train, y_test)

{'Accuracy': 0.6891701828410689,
 'Precision': 0.6703296703296703,
 'Recall': 0.7072463768115942,
 'F1-Score': 0.6882933709449929}

## Logistic Regression

In [44]:
# Define the parameter grid
param_grid = {
    'penalty': ['l1', 'l2'],
    'C': [0.001, 0.01, 0.1, 1, 10, 100],
    'solver': ['liblinear', 'saga']
}

# Create a Logistic Regression classifier
logistic_reg = LogisticRegression()

# Initialize GridSearchCV
grid_search = GridSearchCV(estimator=logistic_reg, param_grid=param_grid, cv=3, scoring='f1', verbose=2, n_jobs=-1)

# Perform grid search
grid_search.fit(X_train_w2v, y_train)

# Print the best parameters and best score
print("Best Parameters:", grid_search.best_params_)
print("Best Score:", grid_search.best_score_)

Fitting 3 folds for each of 24 candidates, totalling 72 fits
Best Parameters: {'C': 1, 'penalty': 'l2', 'solver': 'saga'}
Best Score: 0.7648919191466298


In [45]:
training_and_prediction(LogisticRegression(solver='saga'), X_train_w2v, X_test_w2v, y_train, y_test)

{'Accuracy': 0.7229254571026723,
 'Precision': 0.7055555555555556,
 'Recall': 0.736231884057971,
 'F1-Score': 0.7205673758865249}

## Gradient Boosting

In [47]:
training_and_prediction(XGBClassifier(), X_train_w2v, X_test_w2v, y_train, y_test)

{'Accuracy': 0.7074542897327707,
 'Precision': 0.6897506925207756,
 'Recall': 0.7217391304347827,
 'F1-Score': 0.7053824362606231}

## Deep Learning Techniques with Word2vec

In [48]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, GlobalAveragePooling1D, SpatialDropout1D, Bidirectional, GRU
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping

In [49]:
# Tokenize text data
tokenizer = Tokenizer(num_words=7000)
tokenizer.fit_on_texts(np.concatenate((X_train, X_test), axis=0))
X_train_sequences = tokenizer.texts_to_sequences(X_train)
X_test_sequences = tokenizer.texts_to_sequences(X_test)

In [50]:
high = 0
for temp in X_train_sequences:
  high = max(high, len(temp))
high

125

In [51]:
# Pad sequences to same length
max_len = 150
X_train_padded_sequences = pad_sequences(X_train_sequences, maxlen=max_len)
X_test_padded_sequences = pad_sequences(X_test_sequences, maxlen=max_len)

In [52]:
# Create embedding layer
embedding_dim = 300
embedding_matrix = np.zeros((len(tokenizer.word_index) + 1, embedding_dim))
for word, i in tokenizer.word_index.items():
    if word in model_w2v:
        embedding_matrix[i] = model_w2v[word]
embedding_layer = Embedding(len(tokenizer.word_index) + 1, embedding_dim, weights=[embedding_matrix], trainable=False)

### ANN

In [53]:
def ann_models(num_neurons_layer1, num_neurons_layer2=0, dropout_rate=0):
    model_dense = Sequential()
    model_dense.add(embedding_layer)
    model_dense.add(GlobalAveragePooling1D())
    model_dense.add(Dense(num_neurons_layer1, activation='relu'))
    model_dense.add(Dropout(dropout_rate))
    if num_neurons_layer2 > 0:
      model_dense.add(Dense(num_neurons_layer2, activation='relu'))
    model_dense.add(Dense(1, activation='sigmoid'))
    model_dense.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    model_dense.fit(X_train_padded_sequences, y_train, epochs=100, batch_size=32, verbose=0, callbacks=[early_stop], validation_data=(X_test_padded_sequences, y_test))
    y_pred_dense = model_dense.predict(X_test_padded_sequences)
    y_pred_dense = (y_pred_dense > 0.5).astype(int)
    return [{'num_neurons_layer1': num_neurons_layer1, 'num_neurons_layer2': num_neurons_layer2, 'dropout_rate': dropout_rate}, get_report(y_test, y_pred_dense)]


In [54]:
for num_neurons_layer1 in [128, 64]:
  for num_neurons_layer2 in [64, 32, 0]:
    for dropout_rate in [0.0, 0.1, 0.2, 0.3, 0.4]:
      print(ann_models(num_neurons_layer1, num_neurons_layer2, dropout_rate))

23/23 [==============================] - 0s 3ms/step
[{'num_neurons_layer1': 128, 'num_neurons_layer2': 64, 'dropout_rate': 0.0}, {'Accuracy': 0.7412095639943741, 'Precision': 0.746177370030581, 'Recall': 0.7072463768115942, 'F1-Score': 0.7261904761904763}]
23/23 [==============================] - 0s 3ms/step
[{'num_neurons_layer1': 128, 'num_neurons_layer2': 64, 'dropout_rate': 0.1}, {'Accuracy': 0.7355836849507735, 'Precision': 0.7371601208459214, 'Recall': 0.7072463768115942, 'F1-Score': 0.7218934911242604}]
23/23 [==============================] - 0s 3ms/step
[{'num_neurons_layer1': 128, 'num_neurons_layer2': 64, 'dropout_rate': 0.2}, {'Accuracy': 0.7215189873417721, 'Precision': 0.6949602122015915, 'Recall': 0.7594202898550725, 'F1-Score': 0.7257617728531857}]
23/23 [==============================] - 0s 3ms/step
[{'num_neurons_layer1': 128, 'num_neurons_layer2': 64, 'dropout_rate': 0.3}, {'Accuracy': 0.7341772151898734, 'Precision': 0.7378048780487805, 'Recall': 0.7014492753623188

### LSTM

In [55]:
def lstm_models(num_neurons_layer1, dropout_rate=0):
    model_lstm = Sequential()
    model_lstm.add(embedding_layer)
    model_lstm.add(LSTM(128, dropout=dropout_rate))
    model_lstm.add(Dense(1, activation='sigmoid'))
    model_lstm.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
    model_lstm.fit(X_train_padded_sequences, y_train, epochs=20, batch_size=32, verbose=0, callbacks=[early_stop], validation_data=(X_test_padded_sequences, y_test))
    y_pred_lstm = model_lstm.predict(X_test_padded_sequences)
    y_pred_lstm = (y_pred_lstm > 0.5).astype(int)
    return [{'num_neurons_layer1': num_neurons_layer1, 'dropout_rate': dropout_rate}, get_report(y_test, y_pred_lstm)]

In [56]:
for num_neurons_layer1 in [128, 64]:
  for dropout_rate in [0.0, 0.1, 0.2, 0.3, 0.4]:
    print(lstm_models(num_neurons_layer1, dropout_rate))

23/23 [==============================] - 4s 137ms/step
[{'num_neurons_layer1': 128, 'dropout_rate': 0.0}, {'Accuracy': 0.7172995780590717, 'Precision': 0.7482758620689656, 'Recall': 0.6289855072463768, 'F1-Score': 0.6834645669291338}]
23/23 [==============================] - 3s 93ms/step
[{'num_neurons_layer1': 128, 'dropout_rate': 0.1}, {'Accuracy': 0.7383966244725738, 'Precision': 0.7214484679665738, 'Recall': 0.7507246376811594, 'F1-Score': 0.7357954545454545}]
23/23 [==============================] - 3s 107ms/step
[{'num_neurons_layer1': 128, 'dropout_rate': 0.2}, {'Accuracy': 0.7130801687763713, 'Precision': 0.7210031347962382, 'Recall': 0.6666666666666666, 'F1-Score': 0.6927710843373495}]
23/23 [==============================] - 2s 82ms/step
[{'num_neurons_layer1': 128, 'dropout_rate': 0.3}, {'Accuracy': 0.7215189873417721, 'Precision': 0.754325259515571, 'Recall': 0.6318840579710145, 'F1-Score': 0.6876971608832808}]
23/23 [==============================] - 3s 118ms/step
[{'num_n

### Bidirectional LSTM

In [57]:
def bilstm_models(num_neurons_layer1, spatial_dropout_rate=0, dropout_rate=0):
    model_bilstm = Sequential()
    model_bilstm.add(embedding_layer)
    model_bilstm.add(SpatialDropout1D(spatial_dropout_rate))
    model_bilstm.add(Bidirectional(LSTM(num_neurons_layer1, return_sequences=False)))
    model_bilstm.add(Dropout(dropout_rate))
    model_bilstm.add(Dense(1, activation='sigmoid'))
    model_bilstm.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
    model_bilstm.fit(X_train_padded_sequences, y_train, epochs=10, batch_size=32, verbose=0, callbacks=[early_stop], validation_data=(X_test_padded_sequences, y_test))
    y_pred_bilstm = model_bilstm.predict(X_test_padded_sequences)
    y_pred_bilstm = (y_pred_bilstm > 0.5).astype(int)
    return [{'num_neurons_layer1': num_neurons_layer1, 'spatial_dropout_rate': spatial_dropout_rate, 'dropout_rate': dropout_rate}, get_report(y_test, y_pred_bilstm)]

In [59]:
for num_neurons_layer1 in [128, 64]:
  for spatial_dropout_rate in [0, 0.2]:
    for dropout_rate in [0.1, 0.2, 0.3, 0.4]:
      print(bilstm_models(num_neurons_layer1, spatial_dropout_rate, dropout_rate))

23/23 [==============================] - 4s 159ms/step
[{'num_neurons_layer1': 128, 'spatial_dropout_rate': 0, 'dropout_rate': 0.1}, {'Accuracy': 0.7187060478199718, 'Precision': 0.7474402730375427, 'Recall': 0.6347826086956522, 'F1-Score': 0.6865203761755485}]
23/23 [==============================] - 5s 163ms/step
[{'num_neurons_layer1': 128, 'spatial_dropout_rate': 0, 'dropout_rate': 0.2}, {'Accuracy': 0.7229254571026723, 'Precision': 0.7055555555555556, 'Recall': 0.736231884057971, 'F1-Score': 0.7205673758865249}]
23/23 [==============================] - 5s 182ms/step
[{'num_neurons_layer1': 128, 'spatial_dropout_rate': 0, 'dropout_rate': 0.3}, {'Accuracy': 0.7229254571026723, 'Precision': 0.7055555555555556, 'Recall': 0.736231884057971, 'F1-Score': 0.7205673758865249}]
23/23 [==============================] - 5s 204ms/step
[{'num_neurons_layer1': 128, 'spatial_dropout_rate': 0, 'dropout_rate': 0.4}, {'Accuracy': 0.7229254571026723, 'Precision': 0.7055555555555556, 'Recall': 0.73623

### GRU

In [60]:
model_gru = Sequential()
model_gru.add(embedding_layer)
# model_gru.add(SpatialDropout1D(0.2))
model_gru.add(GRU(64, return_sequences = False))
model_gru.add(Dropout(0.2))
model_gru.add(Dense(1, activation = 'sigmoid'))
model_gru.summary()

Model: "sequential_57"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 300)         3080400   
                                                                 
 gru (GRU)                   (None, 64)                70272     
                                                                 
 dropout_47 (Dropout)        (None, 64)                0         
                                                                 
 dense_107 (Dense)           (None, 1)                 65        
                                                                 
Total params: 3150737 (12.02 MB)
Trainable params: 70337 (274.75 KB)
Non-trainable params: 3080400 (11.75 MB)
_________________________________________________________________


In [61]:
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
model_gru.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model_gru.fit(X_train_padded_sequences, y_train, epochs=50, batch_size=32, validation_data=(X_test_padded_sequences, y_test), callbacks=[early_stop])

Epoch 1/50
89/89 [==============================] - 20s 180ms/step - loss: 0.6510 - accuracy: 0.6112 - val_loss: 0.5806 - val_accuracy: 0.6709
Epoch 2/50
89/89 [==============================] - 14s 155ms/step - loss: 0.5543 - accuracy: 0.7153 - val_loss: 0.5548 - val_accuracy: 0.7131
Epoch 3/50
89/89 [==============================] - 14s 160ms/step - loss: 0.4934 - accuracy: 0.7678 - val_loss: 0.6242 - val_accuracy: 0.7060
Epoch 4/50
89/89 [==============================] - 13s 151ms/step - loss: 0.4787 - accuracy: 0.7730 - val_loss: 0.5359 - val_accuracy: 0.7300
Epoch 5/50
89/89 [==============================] - 13s 150ms/step - loss: 0.4456 - accuracy: 0.7920 - val_loss: 0.5788 - val_accuracy: 0.7117
Epoch 6/50
89/89 [==============================] - 14s 153ms/step - loss: 0.4340 - accuracy: 0.8079 - val_loss: 0.5604 - val_accuracy: 0.7243
Epoch 7/50
89/89 [==============================] - 14s 154ms/step - loss: 0.3960 - accuracy: 0.8265 - val_loss: 0.5560 - val_accuracy: 0.7328

In [62]:
y_pred_gru = model_gru.predict(X_test_padded_sequences)
y_pred_gru = (y_pred_gru > 0.5).astype(int)
get_report(y_test, y_pred_gru)

23/23 [==============================] - 1s 43ms/step


{'Accuracy': 0.729957805907173,
 'Precision': 0.7243401759530792,
 'Recall': 0.7159420289855073,
 'F1-Score': 0.7201166180758019}